---

<h1 style="text-align: center;font-size: 40px;">Estonia Disaster</h1>

---

<center><img src="https://www.marineinsight.com/wp-content/uploads/2014/09/Estonia-ferry-1280x720.jpg" width="500" height="600"></center>

---

>MS Estonia was a cruise ferry built in 1979/80 at the German shipyard Meyer Werft in Papenburg. The ship sank in 1994 in the Baltic Sea in one of the worst maritime disasters of the 20th century.It is the second-deadliest peacetime sinking of a European ship, after the RMS Titanic, and the deadliest peacetime shipwreck to have occurred in European waters, with 852 lives lost.

>Importing necessary Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import missingno as msno
import pandas_profiling
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading the data
df = pd.read_csv("/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv")
df.head()

In [ ]:
df.info()

>Let's see the report of our Dataset at a single Glance

In [ ]:
profile = pandas_profiling.ProfileReport(df)
profile

>Checking Null Values

In [ ]:
n = msno.bar(df,color='gold')

>So we have no Null Values

>Visualization 📊 

In [ ]:
plt.rcParams['figure.figsize'] = (12,9)
#plt.style.use('classic')
color = plt.cm.hsv(np.linspace(0,1,20))
df['Country'].value_counts().plot.bar(color=color)
plt.show()

>Most of the People are from sweden

In [ ]:
plt.rcParams['figure.figsize'] = (10,8)
#plt.style.use('classic')
color = plt.cm.hsv(np.linspace(0,1,20))
sns.countplot(y = 'Country',palette ='Set3',hue='Survived',orient = 'h',data = df)
plt.show()


In [ ]:
plt.rcParams['figure.figsize']=(10,8)
#plt.style.use("classic")
color = ['yellowgreen','gold']
labels =['Not Survived','Survived']
df['Survived'].value_counts().plot.pie(y="Survived",colors=color,explode=(0,0.08),startangle=50,shadow=True,autopct="%0.1f%%")
plt.legend(labels,loc='best')
plt.axis('on');

In [ ]:
plt.rcParams['figure.figsize'] =(9,8)
sns.catplot(x="Sex", hue="Category", col="Survived",
                data=df, kind="count",
                height=6, aspect=.7,palette='Set3')
plt.show()

In [ ]:
facet = sns.FacetGrid(df,hue="Survived",aspect = 4)
facet.map(sns.kdeplot,"Age",shade = True)
facet.set(xlim = (0,df["Age"].max()))
facet.add_legend()
plt.show()

>Most of the People who Survived are Ages between 20 to 40 & most of the people with ages between 45 to 70 could not Survived from the disaster

In [ ]:
plt.rcParams['figure.figsize'] = (10,8)
#plt.style.use('classic')
color = plt.cm.hsv(np.linspace(0,1,20))
sns.countplot(y = 'Country',palette ='Set3',hue='Category',orient = 'h',data = df)
plt.show()

> Most of the Crew People are from Estonia but highest number of Passengers are from Sweden

> Number of Male & Female Survived & Not Survived

In [ ]:
df.groupby(['Survived','Sex'])['Sex'].count()

In [ ]:
df.groupby(['Survived','Sex'])['Sex'].count().plot();

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
plt.style.use("classic")
color = ['yellowgreen','gold','lightskyblue','coral']
df.groupby(['Survived','Sex'])['Sex'].count().plot.pie(colors=color,explode=(0,0.02,0.4,0.01),shadow=True,autopct = '%0.1f%%')
plt.legend(labels,loc='best')
plt.axis('on');

> 46.4% Female could not Survived from the disaster & only 2.7% of the Female could Survived.And 39.7% Male could not Survived from the Disaster & 11.1% Male could Survived.

><h3>Let's see country with number of Passengers</h3>

In [ ]:
Country = pd.DataFrame(df["Country"].value_counts().reset_index().values,columns=["country","NumOfPassenger"])
Country.head()

In [ ]:
import plotly.express as px
fig = px.choropleth(Country,   
    locationmode='country names',
    hover_name="country",
    color='country',
    locations=Country.country,
    hover_data = ['country','NumOfPassenger'],
    featureidkey="Country.NumOfPassenger",
    labels=Country["NumOfPassenger"],
    title= "Country with Number of Passengers"
)
fig.show()


>Feature Engineering

In [ ]:
df.head(2)

>PassengerId,Firstname,Lastname column isn't necessary for our model building,so let's drop this Column

In [ ]:
df.drop(['PassengerId','Firstname','Lastname'],axis=1,inplace=True)

>Mapping the Country

In [ ]:
mapping = {'Sweden': 'Sweden','Estonia':'Estonia','Latvia':'Latvia'}
df['Country'] = df['Country'].map(mapping)
df['Country'].fillna('Other',inplace=True)

In [ ]:
df['Age'] = df['Age'].apply(lambda x: 1 if (x>=0 and x<= 20)
                                                      else (2 if x>20 and x<=40 else (3 if x>40 and x<=60 else 4)))

In [ ]:
df = pd.get_dummies(df,drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split
x = df.drop(['Survived'],axis=1)
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,shuffle=True,test_size=0.2,random_state=0)

>Building our Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model_params  = {
    "decision_tree":{
        "model": DecisionTreeClassifier(),
        "params":{
            'criterion':["entropy","gini"],
            "max_depth":[5,8,9]
        }
    },
    
    "random_forest":{
        "model": RandomForestClassifier(),
        "params":{
            "n_estimators":[1,5,10,15,20,30],#20,30
            "max_depth":[5,8,9,3,2]#3
        }
    },
    
    "adaboost":{
        "model": AdaBoostClassifier(),
        "params":{
            "n_estimators":[1,5,10,15,20,30],
            "learning_rate":[0.01,0.02,0.001,0.03],
            "random_state":[0,100,10]
        }
    }
    
}

In [ ]:
score=[]
for model_name,mp in model_params.items():
    clf = GridSearchCV(mp["model"],mp["params"],cv=8,return_train_score=False)
    clf.fit(x,y)
    score.append({
        "Model" : model_name,
        "Best_Score": clf.best_score_,
        "Best_Params": clf.best_params_
    })
    
df2 = pd.DataFrame(score,columns=["Model","Best_Score","Best_Params"])
df2

>LightGBM

In [ ]:
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(n_estimators=20,learning_rate=0.01,max_depth=3)
lgb.fit(x_train,y_train)
scores=cross_val_score(lgb,x,y,cv=10,scoring="accuracy")
scores.mean()

>KNN

In [ ]:
knn_score=[]
for k in range(1,16):
    knn = KNeighborsClassifier(n_neighbors = k)
    score=cross_val_score(knn,x,y,cv=10)
    knn_score.append(score.mean())
    
knn_score

In [ ]:
%matplotlib inline
plt.figure(figsize=(26,12))
plt.style.use('ggplot')
plt.plot([k for k in range(1,16)],knn_score)
for i in range(1,16):
    plt.text(i,knn_score[i-1],(i,knn_score[i-1]))
    plt.xticks([i for i in range(1,16)])
plt.title("K Neighbor Classifiers Value for different K")
plt.xlabel("k neighbors Value")
plt.ylabel("knn Score")
plt.show()

>We are getting Maximum accuracy at K = 10

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
score=cross_val_score(knn,x,y,cv=10)
score.mean()

>Plotting Confusion Matrix

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score,confusion_matrix
classes_name = ['Not Survived','Survived']
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
mat = confusion_matrix(y_test,y_pred)
plot_confusion_matrix(mat,figsize=(8,6),class_names=classes_name,show_normed=True)
plt.xticks(rotation=0)
plt.show()